К вашей компании пришел заказчик, которому нужно решение задачи анализа тональности отзывов на товары. Заказчик хочет, чтобы вы оценили возможное качество работы такого алгоритма на небольшой тестовой выборке. При этом больше никаких данных вам не предоставляется. Требуется, чтобы качество работы вашего алгоритма (по accuracy) было строго больше 85%.

Оценка качества в этом задании реализована через контест на Kaggle Inclass:

https://inclass.kaggle.com/c/product-reviews-sentiment-analysis

Вам предстоит посмотреть на предоставленные заказчиком отзывы, собрать похожие отзывы в качестве обучающей выборки, и поэкспериментировать с постановкой задачи (разметкой вашей выборки на позитивные и негативные примеры) так, чтобы результат на примерах заказчика был по возможности получше.

Обратите внимание, что заказчик предоставил всего 100 примеров в качестве тестовой выборки - ситуация, когда размеченных данных почти нет - вообще очень частая в индустриальном анализе данных. Конечно, эти отзывы можно было бы идеально разметить вручную и получить максимальное качество, но вы сами не заинтересованы в таком подходе, т.к. потом придется и на всех новых примерах демонстрировать заказчику идеальную работу, что, конечно, вряд ли будет по силам алгоритму. В любом случае рано или поздно алгоритм придется разрабатывать, поэтому попытки "сжульничать" и не делать никакой модели не одобряются.

In [1]:
import time 
import requests
from bs4 import BeautifulSoup
import numpy as np
import json
import csv
import os
import pandas as pd
from sklearn.datasets import load_files


## Получение обучающей выборки

Для формирования обучающей выборки были в ручную отобраны несколько устаревших моделей телефонов с наибольшим количеством отзывов на areviews.ru.

In [38]:
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    'accept' : '*/*'
}
url = 'https://www.eldorado.ru/cat/detail/smartfon-samsung-galaxy-a30s-black-32gb-sm-a307fn/'
file = 'review.csv' 

            
def get_url(url, params=None):
    r = requests.get(url, headers = headers, params=params)
    return r

def get_pages_count(html):
    soup = BeautifulSoup(html, 'html.parser')
    pagination = soup.find_all('a', class_='page')
    if pagination:
        return int(pagination[-1].get_text())
    else:
        return 1

def save_file(items, path):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['rating, review'])
        for item in items:
            writer.writerow([item['rating'],item['Отзыв']])

def get_content(html):
    counter =0 
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_='usersReviewsListItemInnerContainer')
    review = []
    
    for item in items:
        s = item.find('div', class_='middleBlockItem').get_text(
            strip=True).replace('человек считают этот отзыв полезным',' ').replace('\n','').replace('человека считают этот отзыв полезным',' ')
        rating = item.find('div', class_='star starFull').text
        review.append({
            'rating' : rating,
            'Отзыв' : s})
        counter+=1
    return review
    
    
def parse():
    html = get_url(url)
    if html.status_code == 200:
        review = []
        pages_count = get_pages_count(html.text)
        for page in range(1, 11):
            print (f'Парсинг страницы {page} из ...')
            html = get_url(url+'/page/'+str(page)+'?show=response')
            review.extend(get_content(html.text))
            #review = get_content(html.text)
        save_file(review, file)
                
        print(f'Получено {len(review)} отзывов')
        
    else:
        print ('error')
        

parse()

Парсинг страницы 1 из ...
Парсинг страницы 2 из ...
Парсинг страницы 3 из ...
Парсинг страницы 4 из ...
Парсинг страницы 5 из ...
Парсинг страницы 6 из ...
Парсинг страницы 7 из ...
Парсинг страницы 8 из ...
Парсинг страницы 9 из ...
Парсинг страницы 10 из ...
Получено 100 отзывов


In [24]:
data = pd.read_csv('review.csv', names=['review'],sep=';', encoding='cp1251',header=0)
data

,review
0,Покупал в подарок дочери на день рождения .Тел...
1,Пока все нравится. За приемлемую цену неплохой...
2,За свои деньги хороший смартфон. Работает шуст...
3,Отличный телефон! Все приложения и игры летают...
4,"Телефон отличный,компактный . Батарея держит д..."
...,...
5,Хороший смартфон по разумной цене. До 2 дней м...
6,Покупал в подарок крестнику на НГ.Телефон отли...
7,Купила смартфон ребёнку. Достойная модель за п...
8,"Купили сыну,ему пока все нравится!Сказал класс..."


In [4]:
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    'accept' : '*/*'
}
url = 'https://market.yandex.ru/product--telefon-samsung-star-gt-s5230/4578928/reviews?track=tabs&page='
file = 'review_yandex.csv'

def get_url(url, params=None):
    r = requests.get(url, headers = headers, params=params)
    return r

def save_file(items, path):
    with open(path, 'a', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['review', 'counter'])
        for item in items:
            writer.writerow([item['Отзыв'],item['counter']])
            

def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('dl', class_='_27K1Pkf_Bb')
    review = []
    counter = 0
    for item in items:
        time.sleep(15)
        s1 = (item.find_all('dd', class_='_1yULRh4B4H jwsQ_1V8jD'))
        
        for s in s1:
            r = s.get_text(strip=True).replace('\n','').replace('\r','')
            review.append({"Отзыв" : r, 'counter' : counter})
            counter +=1
    return review
    
    
    
def parse():
    html = get_url(url)
    if html.status_code == 200:
        
        review = []
        for page in range(2, 11):
            time.sleep(15)
            print (f'Парсинг страницы {page} из ...')
            html = get_url(url+str(page))
            review.extend(get_content(html.text))
        print(f'Получено {len(review)} отзывов')
        print (review)
        save_file(review, file)   
    else:
        print ('error')
   
        
        
        
parse()

Парсинг страницы 2 из ...
Парсинг страницы 3 из ...
Парсинг страницы 4 из ...
Парсинг страницы 5 из ...
Парсинг страницы 6 из ...
Парсинг страницы 7 из ...
Парсинг страницы 8 из ...
Парсинг страницы 9 из ...
Парсинг страницы 10 из ...
Получено 0 отзывов
[]


In [53]:
%%time
# review parsing price&reviews
# Счетчик отзывов
counter = 0
# Загружаем отобранные ссылки из файла
links = ['http://areviews.ru/products/lg_l80_d380/reviews', 'http://areviews.ru/products/htc_windows_phone_8s/reviews']
for link in links:
    # На всякий случай делаем случайную задержку перед каждым парсингом
    t_sleep = np.random.randint(1, 100)*0.01
    time.sleep(t_sleep)
    # читаем данные по ссылке
    req = requests.get(link)
    link_parser = BeautifulSoup(req.text, 'lxml')
    # Находим все отзывы на странцие
    reviews_parser = link_parser.find('ul', attrs={'class', 'comment_list'})
    # Парсим следующий отзыв
    review_parser = reviews_parser.findAll('li')
    for review in review_parser:
        # Находим заголовок отзыва и извлекаем из него рейтинг, который был поставлен товару
        pos = []
        neg = []
        comment_header = review.find('div', attrs={'class', 'comment_header'})
        if not comment_header:
            continue
        rating = 5 - len(comment_header.findAll('i', attrs={'class', 'empty-star'}))
        # Находим текст отзыва, выделяем Достоинства, Недостатки и Комментарий
        review_parts = review.findAll('p')
        for res in review_parts:
            counter += 1
            
            if res.findChild('b').text == u"Достоинства:":
                # Рассматриваем отмеченные достоинства как отдельный положительный отзыв
                # Записываем его в папку ./pos
                pos.extend(res.text)
            with open('pos.txt','w') as f:
                f.write(str(pos).strip())
            if res.findChild('b').text == u"Недостатки:":
                # Рассматриваем отмеченные недостатки как отдельный негативный отзыв
                # Записываем его в папку ./neg
                neg.extend(res.text)
            
            with open ('neg.txt','w') as f:
                f.write(str(neg))
          

Wall time: 5.42 s


In [76]:
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
    'accept' : '*/*'
}
url = 'http://areviews.ru/products/lg_l80_d380/reviews'
file = 'areviews.csv' 

            
def get_url(url, params=None):
    r = requests.get(url, headers = headers, params=params)
    return r


def save_file(items, path):
    with open(path, 'w', newline='') as file:
        writer = csv.writer(file, delimiter=';')
        writer.writerow(['label, rating, review'])
        for item in items:
            writer.writerow([item['label'],item['rating'],item['Отзыв']])

def get_content(html):
    review = [] 
    label = 0
    soup = BeautifulSoup(html, 'html.parser')
    reviews_parser = soup.find('ul', attrs={'class', 'comment_list'})
    items = reviews_parser.find_all('li')
    for item in items:
        comment_header = item.find('div', attrs={'class', 'comment_header'})
        if not comment_header:
            continue
        rating = 5 - len(comment_header.findAll('i', attrs={'class', 'empty-star'}))
        review_parts = item.findAll('p')
        for res in review_parts:
            if res.findChild('b').text == u"Достоинства:":
                r =  res.text[12:].strip().replace('\n',' ')
                label = 1
            if res.findChild('b').text == u"Недостатки:":
                r = res.text[11:].strip().replace('\n',' ')
                label = 0
            if res.findChild('b').text == u"Комментарий:":
                if rating > 3:
                    r = res.text[11:].strip().replace('\n',' ')
                    label = 1
                else:
                    r = res.text[12:].strip().replace('\n',' ')
                    label = 0
            review.append({
                'label' : label,
                'rating' : rating,
                'Отзыв' : r})
        
    return (review)
    
    
def parse():
    html = get_url(url)
    if html.status_code == 200:
        review = get_content(html.text)
        save_file(review, file)
                
        print(f'Получено {len(review)} отзывов')
        
    else:
        print ('error')
        

parse()

Получено 177 отзывов


In [80]:
data = pd.read_csv('areviews.csv', names=['label','rating','review'],sep=';', encoding='cp1251',header=0)
data

,label,rating,review
0,1,3,"Большой экран, приятный внешний вид."
1,0,3,Нестабилен в работе
2,0,3,Телефон неплох. Но после года использования на...
3,1,3,Яркий и красочный экран
4,0,3,Программное обеспечение
...,...,...,...
172,0,5,"Из недостатков выделю то, что половина из 4Gb ..."
173,1,5,": Раньше я боялась андройда, думала, что не ос..."
174,1,5,1. Диагональ экрана 5`` и хорошая яркость. 2. ...
175,0,5,1. Регулировка яркости осуществляется вручную ...
